In [1]:
!pip install bs4
!pip install datetime
!pip install requests
!pip install beautifulsoup
!pip install selenium

  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [19 lines of output]
      Traceback (most recent call last):
        File "/home/kader/Descargas/Proyecto-empleo-main/.venv/lib64/python3.9/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 353, in <module>
          main()
        File "/home/kader/Descargas/Proyecto-empleo-main/.venv/lib64/python3.9/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
        File "/home/kader/Descargas/Proyecto-empleo-main/.venv/lib64/python3.9/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 118, in get_requires_for_build_wheel
          return hook(config_settings)
        File "/tmp/pip-build-env-nd0li5q3/overlay/lib/python3.9/sit

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
from time import sleep
import requests


# Inicializar driver de Chrome
driver = webdriver.Chrome()

# Navegamos a la página web
driver.get("https://www.jobleads.com/es")

# Aceptamos el banner de coookies
accept_cookies_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Aceptar")]'))  # Modifica el XPATH según sea necesario
)

# Hacemos clic en el botón de "Aceptar cookies"
accept_cookies_button.click()

# Encontramos el botón de iniciar sesión y hacemos click
login_button = driver.find_element(By.XPATH, '//*[@id="__nuxt"]/div[1]/div[1]/div[1]/div/div/div[2]/button')
login_button.click()

# Esperamos hasta que el campo de correo electrónico esté presente
email_input = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, '//input[@type="email" and @data-testid="login-form-email-field-ui-input"]'))
)
email_input.send_keys("cakapa61@gmail.com")

password_input = WebDriverWait(driver, 3).until(
    EC.presence_of_element_located((By.XPATH, '//input[@type="password" and @data-testid="login-form-password-field-ui-input"]'))  # Cambia el data-testid si es necesario
)
# Ingresa la contraseña
password_input.send_keys("DaB01.proyecto")

# Si tienes un botón de inicio de sesión, puedes hacer clic en él
login_button = WebDriverWait(driver, 3).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[4]/div/div/form/div[3]/button'))
)
login_button.click()

# Ya hemos conseguido iniciar sesión para acceder a la información oculta, ahora intentemos extraer dicha información
# Para ello, debemos ir al área de "Trabajos" y desde ahí buscar el resto

try:
    trabajos_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//a[@data-testid="app-header-jobs-menu-item"]'))
    )
    
    # Hacemos clic en el botón "Trabajos"
    trabajos_button.click()
except StaleElementReferenceException:
    
    # Reencontramos el botón y hacer clic de nuevo
    trabajos_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//a[@data-testid="app-header-jobs-menu-item"]'))
    )
    trabajos_button.click()
    pass

# Definir las palabras clave y ubicaciones
keywords = ["Analyst", "Data Scientist", "Data Engineer"]
locations = ["Madrid", "Barcelona"]

# Lista para almacenar las URLs de trabajos
job_urls = []

# Iterar sobre las combinaciones de keywords y locations
for keyword in keywords:
    for location in locations:
        for num in range(1,8):
        # Construir la URL de búsqueda con los parámetros deseados
            url = f"https://www.jobleads.com/search/jobs?keywords={keyword}&location={location}&location_country=ESP&page={num}"

            # Hacer la solicitud a la URL
            driver.get(url)

            # Esperar a que la página se cargue y extraer URLs
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'h2')))

            soup1 = BeautifulSoup(driver.page_source, "html.parser")

            # Extraer las URLs de cada oferta
            job_urls.extend(
                [f"https://jobleads.com{x['href']}" for x in soup1.find_all("a", class_="absolute left-0 right-0 top-0 bottom-0 opacity-0 z-[1] cursor-pointer")]
            )

# Lista para almacenar los datos de posiciones
posiciones_data = []

# Extraer información de cada URL
for url in job_urls:
    driver.get(url)
    
    # Esperar a que la página se cargue
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'h5')))
    
    soup1 = BeautifulSoup(driver.page_source, "html.parser")
    
    # Extraemos datos
    title = soup1.find('h5', class_='text-xl font-bold text-black').text if soup1.find('h2', class_='text-xl font-bold text-black') else 'Sin título'
    
    location = soup1.find('div', class_='py-1 rounded-full bg-[#f4f4f4] text-tp px-3 text-sm').text if soup1.find('div', class_='py-1 rounded-full bg-[#f4f4f4] text-tp px-3 text-sm') else 'Sin ubicación'
    
    description = soup1.find('p', class_='mt-3 text-base text-[#262D37]').text if soup1.find('p', class_='mt-3 text-base text-[#262D37]') else 'Sin descripción'
    
    servicios = [servicio.text for servicio in soup1.find_all('div', class_='flex gap-2 flex-wrap mt-3')] if soup1.find_all('div', class_='flex gap-2 flex-wrap mt-3') else []

    timestamp_extraccion = pd.Timestamp.now()
    # En el caso del sueldo, encontramos una complicación ya que aporta un rango y no siempre se encuentra en la misma posición, por lo que tenemos que realizar un pequeño apaño antes de descargar
    # En una primera instancia, ponemos como predefinido que no aparezca el salario
    sueldo = 'Sin sueldo'
    # Ahora que tenemos algo por defecto, intentemos sortear el código
    divs = soup1.find_all('div', class_='py-1 rounded-full bg-[#f4f4f4] text-tp px-3 text-sm')
    for div in divs:
        if "EUR" in div.text:
            sueldo = div.text
    # Añadir la posición a la lista
    posiciones_data.append({
        'Título': title,
        'Ubicación': location,
        'Descripción': description,
        'Servicios': servicios,
        'Sueldo':sueldo,
        'Timestamp': timestamp_extraccion,
        'Url':url
    })


# Cerramos el controlador
driver.quit()

# Ahora que tenemos datos, tenemos que crear un df

df = pd.DataFrame(posiciones_data)

print(df)

df.to_csv('portales_empleo.csv', index=True)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="__nuxt"]/div[1]/div[1]/div[1]/div/div/div[2]/button"}
  (Session info: chrome=131.0.6778.69); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x56350524c4ca <unknown>
#1 0x563504d5f620 <unknown>
#2 0x563504dae306 <unknown>
#3 0x563504dae5a1 <unknown>
#4 0x563504df3a04 <unknown>
#5 0x563504dd239d <unknown>
#6 0x563504df0de0 <unknown>
#7 0x563504dd2113 <unknown>
#8 0x563504da0be0 <unknown>
#9 0x563504da1bbe <unknown>
#10 0x563505218e4b <unknown>
#11 0x56350521cde2 <unknown>
#12 0x563505205d2c <unknown>
#13 0x56350521d957 <unknown>
#14 0x5635051eb4bf <unknown>
#15 0x56350523b348 <unknown>
#16 0x56350523b510 <unknown>
#17 0x56350524b346 <unknown>
#18 0x7f8cc0689d82 start_thread


In [ ]:
# ahora que tenemos datos, tenemos que crear un df

df = pd.DataFrame(posiciones_data)

In [ ]:
posiciones_data

[{'Título': 'Sin título',
  'Ubicación': 'Madrid',
  'Descripción': 'Join a leading provider of credit ratings as a Senior Analyst in Structured Credit. This exciting role involves analyzing complex financial transactions, preparing rating proposals, and communicating insights to investors. You will be part of a dynamic team that plays a crucial role in maintaining the integrity of the credit markets. The ideal candidate will possess strong analytical skills, advanced proficiency in Excel, and the ability to review legal documents from a credit risk perspective. Embrace the opportunity to make a significant impact in a collaborative environment that values diverse perspectives and innovative thinking.',
  'Servicios': [' Credit Analysis Cash Flow Modelling Securitisation Structures Legal Document Analysis Advanced Excel Skills Communication Skills',
   ' Degree in Finance or related field Experience in finance'],
  'Sueldo': 'EUR 50,000 - 70,000',
  'Timestamp': Timestamp('2024-11-14 1

In [ ]:
df

0       https://jobleads.com/job/eba2bf38bdc4c66ba2f5f...
1       https://jobleads.com/job/e4b1388ab13374dab4103...
2       https://jobleads.com/job/ec2be137e14f9dbc1a8eb...
3       https://jobleads.com/job/ea5922eccaee3331c3aba...
4       https://jobleads.com/job/ea6ecb7b732973b55d2da...
                              ...                        
1017    https://jobleads.com/job/e09a569bd0aaa37bf0411...
1018    https://jobleads.com/job/e6c30361e7618acf8e876...
1019    https://jobleads.com/job/e095a7fdb6c3866944e2c...
1034    https://jobleads.com/job/ee55bb8c0f23269699759...
1035    https://jobleads.com/job/e4ac3a99eaaa5eaa22c82...
Name: Url, Length: 780, dtype: object